# CEE 598: Uncertainty Quantification
## Assignment Six
### Noah Garfinkle (garfink2@illinois edu)
### 03 May 2020

## Imports and Set Up

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import pytest
import chaospy as cp
import pandas as pd

## References

In [2]:
# https://chaospy.readthedocs.io/en/master/tutorial.html
